In [23]:
# imports 
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
from scipy.optimize import minimize

npa= np.array

In [24]:
#define parameters bounds 
alpha_bnd = (0,1) #learning rate
beta_bnd = (0, 100) #inverse temperature
rho_bnd = (-1.5,1.5) #threshold (only for foraging model)


epsilon = 1e-10 #to avoid log underflow 

In [25]:
#download data sample named "Experiment1_data" and change dataPath to your local path 
dataPath = "Experiment1_data.pickle"

#extract data
with open(dataPath, 'rb') as handle:
    all_sub_2ab = pickle.load(handle)

In [41]:
subs = all_sub_2ab.keys()

In [42]:
excluded_subs = []

for sub in tqdm(subs) : 
    df= all_sub_2ab[sub]
    df= df.drop(df.index[0:25])
    switch = np.array((df['choice'] != df['choice'].shift()).cumsum())[-1]

    #exclude partcipants that didn't switch at least once
    if switch < 2 : 
        excluded_subs.append(sub) 
    
subs = [sub for sub in subs if sub not in excluded_subs]
    

100%|██████████| 40/40 [00:00<00:00, 1878.39it/s]


## Define models 


In [43]:
def LSE(x): 
    x = np.array(x)
    c=np.max(x)
    return c + np.log(np.sum(np.exp(x - c)))

### Reinforcement learning

In [44]:
def ql_model(theta,df):

    sideList= df['choice'].tolist()
    rewardList = df['reward'].tolist()
    
    karm= max(sideList)+1
    Qvalue = np.zeros(karm)+(1/karm) 

    alpha = theta[0]
    beta = theta[1]

    probList=[]

    for i in range (len(rewardList)):
        arm = int(sideList[i]) 

        
        ##Softmax function
        prob = np.exp((Qvalue[arm]*beta)-LSE(Qvalue*beta))
        
        probList.append(prob)
        Qvalue [arm] += alpha*(rewardList[i]-Qvalue[arm])

        ##log likelihood
    logLike = (np.sum(np.log(npa(probList)+epsilon))) * (-1)
    
    return logLike

### Foraging

In [45]:
def forage_model(theta,df):  

    
    sideList= df['choice'].tolist()
    rewardList = df['reward'].tolist()
    
   
    oitOreList= []

    #switch/explore = 0, stay/exploit = 1
    for j in range (len(sideList)-1):  
        if sideList[j]==sideList[j+1]:
            oitOreList.append(1)
        else:
            oitOreList.append(0)

    rewardList = rewardList[1:]
    
    

    alpha = theta[0]
    beta = theta[1]   
    rho = theta[2]

    v_oit = 1

    probList=[]
    for i in range (len(rewardList)):
        oit = oitOreList[i] # oit or ore 

        if oit == 1:
            prob = np.exp((v_oit*beta)-LSE([v_oit*beta, rho*beta]))
            probList.append(prob)
            v_oit += alpha*(rewardList[i]-v_oit)

        if oit == 0:
            prob = 1-(np.exp((v_oit*beta)-LSE([v_oit*beta, rho*beta])))
            probList.append(prob)
            v_oit = rho + alpha*(rewardList[i]-rho) 
    

    ##log likelihood
    logLike = (np.sum(np.log(npa(probList)+epsilon))) * (-1)
  
    return logLike

## 3. Optimisation functions

In [46]:
def fitQL(sub,df, replication) : 
    nll = []
    alphaList = []
    betaList = []

    succ_list = []
    msg_list = []

    bnds = (alpha_bnd,beta_bnd )
    
    for h in range (replication):
        
    
        alpha = np.random.random()
        beta = np.random.exponential(scale =1, size = None)
        startParams = [alpha, beta]
        theta_optim = minimize(lambda startParams : ql_model(startParams,df), startParams, 
                               method = "Nelder-Mead", bounds = bnds,
                               options={'maxfev': 1000,'maxiter': 1000, 'xatol':0.001, 'fatol': 0.001 } )
        
        if h == 0:
            minimizeLog = theta_optim.fun
            optimX = theta_optim.x
        else:
            if theta_optim.fun < minimizeLog:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x

        succ = theta_optim.success
        mgs = theta_optim.message

    nll.append(minimizeLog)
    theta =  (optimX.tolist())
    alphaList.append(theta[0])
    betaList.append(theta[1])
    succ_list.append(succ)
    msg_list.append(mgs)
    dataDict = {'sub': sub ,'alpha': alphaList, 'beta': betaList,  'likelihood': nll, 'succ':succ_list, 'msg':msg_list}
    opt_QL = pd.DataFrame(dataDict)
    return opt_QL


In [47]:
def fitFOR(sub,df,replication) :

    nll = []
    alphaList = []
    betaList = []
    rhoList = []
    
    bnds = (alpha_bnd,beta_bnd,rho_bnd )
    
    succ_list = []
    msg_list = []

    for h in range (replication):
        
        alpha = np.random.random()
        beta = np.random.exponential(scale =1, size = None)
        rho = np.random.uniform(rho_bnd[0], rho_bnd[1])
        

        startParams = [alpha, beta,rho]
        theta_optim = minimize(lambda startParams : forage_model(startParams,df), startParams, 
                               method = "Nelder-Mead", bounds = bnds, 
                               options={'maxfev': 1000,'maxiter': 1000, 'xatol':0.001, 'fatol': 0.001})

        if h == 0:
            minimizeLog = theta_optim.fun
            optimX = theta_optim.x
        else:
            if theta_optim.fun < minimizeLog:
                
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
                
        succF = theta_optim.success
        mgs_F = theta_optim.message

    nll.append(minimizeLog)
    theta =  (optimX.tolist())
    alphaList.append(theta[0])
    betaList.append(theta[1])
    rhoList.append(theta[2])
    succ_list.append(succF)
    msg_list.append(mgs_F)

    dataDict = { 'sub':sub, 'alpha': alphaList, 'beta': betaList, 'threshold': rhoList, 
                'likelihood': nll,'succ':succ_list, 'msg':msg_list}
    opt_FOR = pd.DataFrame(dataDict)
    return opt_FOR
    

## Model fitting

The final files contains fitted parameters by maximum likelihood for the sample data. 

You can visualize the likelihoods and AIC by downloading and running the "FigureFitExperiment1" jupyter notebook.


In [48]:
opt_QL = pd.DataFrame()
opt_FOR = pd.DataFrame()

for sub in tqdm(subs) : 
    n_rep=50
    df= all_sub_2ab[sub]
    df= df.drop(df.index[0:25])
    data = fitFOR(sub,df,n_rep)
    opt_FOR = pd.concat([opt_FOR,data], axis= 0)
    data = fitQL(sub,df,n_rep)
    opt_QL = pd.concat([opt_QL,data], axis= 0)


100%|██████████| 37/37 [12:53<00:00, 20.91s/it]


It is important to note that the optimization algorithm may fail to converge, for instance, due to exceeding the allowed number of iterations. To save time, all failed convergence cases will be handled in the next notebook.

In [55]:
opt_QL

,sub,alpha,beta,likelihood,succ,msg
0,10,0.596581,10.869895,24.920992,True,Optimization terminated successfully.
0,11,0.656768,9.195482,20.207209,True,Optimization terminated successfully.
0,12,0.930580,3.718974,82.530820,True,Optimization terminated successfully.
0,13,0.120916,5.412621,70.809498,True,Optimization terminated successfully.
0,14,0.734711,3.577618,86.109271,True,Optimization terminated successfully.
0,15,0.459309,3.761062,104.120235,True,Optimization terminated successfully.
0,16,0.278418,20.554649,8.024894,True,Optimization terminated successfully.
0,18,0.002865,100.000000,20.572512,True,Optimization terminated successfully.
0,19,0.002728,100.000000,26.772317,True,Optimization terminated successfully.
0,20,0.558529,7.549170,33.844447,True,Optimization terminated successfully.


In [36]:
#Save 
opt_FOR.to_csv('ForagingParams.csv')
opt_QL.to_csv('RLParams.csv')